In [2]:
#imports
import pandas as pd

from tqdm import tqdm
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *

Preprocessing of our data (creating smaller subset)

In [ ]:
def split_file_by_lines(filename, lines_per_file):
    with open(filename, 'r', encoding='utf-8') as file: 
        count = 0
        current_file = open('data/wikipediaSubset.txt', 'w', encoding='utf-8') 
        
        numLine = 0
        count = 0
        for line in file:
            if(numLine > lines_per_file):
                break
            if (count % 1000) == 0:
                current_file.write(line)
                numLine += 1
            count += 1    


        current_file.close()

In [3]:
def txt_to_df(filename):
    df = pd.read_csv(filename, delimiter=' \|\|\| ', engine='python', header=None, names=['Title', 'Text'])
    return df

In [3]:
#split_file_by_lines('data/stemmed.txt', 1000)
df = txt_to_df('data/wikipediaSubset.txt')
display(df)

C:\Users\Jakob\AppData\Local\Temp\ipykernel_11940\3461223430.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(filename, sep="|||")


ParserError: Expected 848 fields in line 3, saw 1277. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.

Preprocesses the data generated by the LLM in the same way as the original data (https://github.com/tscheepers/Wikipedia-Summary-Dataset/blob/master/src/process.py)

In [6]:
def file_iterator():
    with open('LLM-Output.txt', 'r') as file:
        for line in file:
            split = line.split("|||")
            title = split[0]
            #join in the rare case this sequenze occurs more than once
            text = ''.join(map(str, split[1:]))
            yield (title, text)

def preprocess_LLM_output():
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    punctiation_words = ['.', ',', ';', ':', '(', ')', '`', '\'', '\'\'', '-', '–', '—', '…', '[', ']', '...', '{', '}']
    forbidden_words = ['']
    punctiation_chars = ['`']
    df = {'title': [], 'text': [] }
    for (title, text) in file_iterator():

        #Tokenize
        title = word_tokenize(title)
        text = word_tokenize(text)

        #convert words to lowercase
        title = [t.lower() for t in title]
        text = [t.lower() for t in text]

        #remove punctuation
        title = [t for t in title if t not in punctiation_words]
        text = [t for t in text if t not in punctiation_words]

        #remove punctuation chars
        title = [''.join(c for c in t if c not in punctiation_chars) for t in title]
        text = [''.join(c for c in t if c not in punctiation_chars) for t in text]

        #remove forbidden words
        title = [t for t in title if t not in forbidden_words]
        text = [t for t in text if t not in forbidden_words]

        #remove stopwords
        title = [t for t in title if t not in stopwords_english]
        text = [t for t in text if t not in stopwords_english]

        #stemm
        title = [stemmer.stem(t) for t in title]
        text = [stemmer.stem(t) for t in text]

        df['title'].apped(title)
        df['text'].append(text)

